<a href="https://colab.research.google.com/github/HassanSuhaib71/Task-5-Mental-Health-Support-Chatbot-Fine-Tuned-/blob/main/Task_5_Mental_Health_Support_Chatbot_(Fine_Tuned).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opendatasets pandas


In [3]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/atharvjairath/empathetic-dialogues-facebook-ai")

Skipping, found downloaded files in "./empathetic-dialogues-facebook-ai" (use force=True to force download)


In [4]:
import pandas as pd
data = pd.read_csv("/content/empathetic-dialogues-facebook-ai/emotion-emotion_69k.csv")
data.head()

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels,Unnamed: 5,Unnamed: 6
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju...",NaN,NaN
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?,NaN,NaN
2,2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...,NaN,NaN
3,3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.,NaN,NaN
4,4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.,NaN,NaN


In [5]:
!pip install transformers datasets evaluate accelerate


In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling


In [7]:
dataset = load_dataset("csv", data_files="/content/empathetic-dialogues-facebook-ai/emotion-emotion_69k.csv")
dataset = dataset["train"].train_test_split(test_size=0.1)


In [8]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./model_cache")
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="./model_cache")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [10]:
def tokenize_function(example):
    prompts = ["User: " + s + "\nTherapist:" for s in example["Situation"]]
    responses = example["empathetic_dialogues"]
    full_texts = [p + " " + r for p, r in zip(prompts, responses)]
    return tokenizer(full_texts, truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/58172 [00:00<?, ? examples/s]

Map:   0%|          | 0/6464 [00:00<?, ? examples/s]

In [11]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [12]:
training_args = TrainingArguments(
    output_dir="./mental_health_bot",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
)


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-13-1223269587.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
trainer.save_model("fine_tuned_mental_health_bot")
tokenizer.save_pretrained("fine_tuned_mental_health_bot")


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("fine_tuned_mental_health_bot")
model = AutoModelForCausalLM.from_pretrained("fine_tuned_mental_health_bot")

chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    prompt = f"User: {user_input}\nTherapist:"
    response = chatbot(prompt, max_new_tokens=50, do_sample=True, top_k=50)[0]["generated_text"]
    print("Bot:", response.replace(prompt, "").strip())
